In [1]:
import numpy as np 
import pandas as pd
import time 

In [2]:
# il dataset è direttamente quello ottenuto dal notebook preprocessing tramite i filtri che abbiamo implementato 
dataset = pd.read_csv("dati_LS.csv")
display(dataset)

,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME,DETECTOR,LAYER
0,13.0,1.897417e+09,445.0,24.0,11145.000000,3.0,4.0
1,14.0,1.897417e+09,450.0,9.0,11257.500000,3.0,2.0
2,16.0,1.897417e+09,452.0,17.0,11314.166667,3.0,1.0
3,139.0,1.897417e+09,454.0,0.0,11350.000000,0.0,0.0
4,15.0,1.897417e+09,458.0,9.0,11457.500000,3.0,3.0
...,...,...,...,...,...,...,...
122545,55.0,1.920940e+09,325.0,4.0,8128.333333,3.0,3.0
122546,56.0,1.920940e+09,327.0,22.0,8193.333333,3.0,1.0
122547,139.0,1.920940e+09,329.0,0.0,8225.000000,0.0,0.0
122548,54.0,1.920940e+09,331.0,20.0,8291.666667,3.0,2.0


In [12]:
#per qualche motivo non funziona
df = dataset.copy()
df["DRIFT_TIME"] = np.zeros(len(df))
df["HOR_AMBIG_COORD"] = np.zeros(len(df))
#display(df)
#df.dtypes

def compute_drift_time(x) :
    v = 42/(2*390) #drift velocity in mm/ns
    t0 = x.TIME[x[x['DETECTOR'] == 0].index]
    l = (x[x['DETECTOR'] != 0]).index 
    # abbandono l'idea di nan assegnando array di zeri alle colonne drift time e hor_ambig_coord
    x.loc[l, 'DRIFT_TIME'] = np.abs(np.array(x.loc[l, 'TIME'])-np.array(t0))
    x.HOR_AMBIG_COORD[l] = v*np.abs(x.TIME[l] - t0)
    return x

df = df.groupby(['ORBIT_CNT'],group_keys=False)\
                     .apply(compute_drift_time)\
                     .reset_index()\
                     .drop(['index'],axis=1) #niente dropna per non perdere i trigger
df

,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME,DETECTOR,LAYER,DRIFT_TIME,HOR_AMBIG_COORD
0,13.0,1.897417e+09,445.0,24.0,11145.000000,3.0,4.0,0.0,0.0
1,14.0,1.897417e+09,450.0,9.0,11257.500000,3.0,2.0,0.0,0.0
2,16.0,1.897417e+09,452.0,17.0,11314.166667,3.0,1.0,0.0,0.0
3,139.0,1.897417e+09,454.0,0.0,11350.000000,0.0,0.0,0.0,0.0
4,15.0,1.897417e+09,458.0,9.0,11457.500000,3.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
122545,55.0,1.920940e+09,325.0,4.0,8128.333333,3.0,3.0,0.0,0.0
122546,56.0,1.920940e+09,327.0,22.0,8193.333333,3.0,1.0,0.0,0.0
122547,139.0,1.920940e+09,329.0,0.0,8225.000000,0.0,0.0,0.0,0.0
122548,54.0,1.920940e+09,331.0,20.0,8291.666667,3.0,2.0,0.0,0.0


In [38]:
dfprova = df.loc[df.groupby(['ORBIT_CNT']).groups.get(1897417007.0)]
v = 42/(2*390)
trigger_index = dfprova[dfprova['DETECTOR'] == 0].index # indice del trigger 
t0 = dfprova.TIME[trigger_index]
l = (dfprova[dfprova['DETECTOR'] != 0]).index 
dfprova.loc[l, 'DRIFT_TIME'] = np.abs(np.array(dfprova.loc[l, 'TIME'])-np.array(t0))
dfprova 

,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME,DETECTOR,LAYER,DRIFT_TIME,HOR_AMBIG_COORD
0,13.0,1.897417e+09,445.0,24.0,11145.000000,3.0,4.0,205.000000,0.0
1,14.0,1.897417e+09,450.0,9.0,11257.500000,3.0,2.0,92.500000,0.0
2,16.0,1.897417e+09,452.0,17.0,11314.166667,3.0,1.0,35.833333,0.0
3,139.0,1.897417e+09,454.0,0.0,11350.000000,0.0,0.0,0.000000,0.0
4,15.0,1.897417e+09,458.0,9.0,11457.500000,3.0,3.0,107.500000,0.0


In [11]:
#non capisco!!
df = dataset.copy()
df["DRIFT_TIME"] = np.nan
df["HOR_AMBIG_COORD"] = np.nan
#display(df)
#df.dtypes

def assign_time_pedestal(group) :
    t0 = group["TIME"][group.DETECTOR == 0] #.astype(float)
    #print(type(t0))
    group1 = group#.copy() #è colpa di view vs copy? non sembra
    group1["TIME_PEDESTAL"] = t0 
    return group1

df = df.groupby(['ORBIT_CNT'],group_keys=False)\
                     .apply(assign_time_pedestal)\
                     .reset_index()\
                     .drop(['index'],axis=1) #niente dropna per non perdere i trigger
df

,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME,DETECTOR,LAYER,DRIFT_TIME,HOR_AMBIG_COORD,TIME_PEDESTAL
0,13.0,1.897417e+09,445.0,24.0,11145.000000,3.0,4.0,NaN,NaN,NaN
1,14.0,1.897417e+09,450.0,9.0,11257.500000,3.0,2.0,NaN,NaN,NaN
2,16.0,1.897417e+09,452.0,17.0,11314.166667,3.0,1.0,NaN,NaN,NaN
3,139.0,1.897417e+09,454.0,0.0,11350.000000,0.0,0.0,NaN,NaN,11350.0
4,15.0,1.897417e+09,458.0,9.0,11457.500000,3.0,3.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
122545,55.0,1.920940e+09,325.0,4.0,8128.333333,3.0,3.0,NaN,NaN,NaN
122546,56.0,1.920940e+09,327.0,22.0,8193.333333,3.0,1.0,NaN,NaN,NaN
122547,139.0,1.920940e+09,329.0,0.0,8225.000000,0.0,0.0,NaN,NaN,8225.0
122548,54.0,1.920940e+09,331.0,20.0,8291.666667,3.0,2.0,NaN,NaN,NaN


In [15]:
# df = dataset.copy()
# df["DRIFT_TIME"] = np.nan #nota: grazie ai filtri precedenti so che ogni orbita ha esattamente un trigger (ordine 0)
# t0 = df.TIME[df.DETECTOR == 0]
# df["DRIFT_TIME"] = np.nan


,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME,DETECTOR,LAYER
0,13.0,1.897417e+09,445.0,24.0,11145.000000,3.0,4.0
1,14.0,1.897417e+09,450.0,9.0,11257.500000,3.0,2.0
2,16.0,1.897417e+09,452.0,17.0,11314.166667,3.0,1.0
3,139.0,1.897417e+09,454.0,0.0,11350.000000,0.0,0.0
4,15.0,1.897417e+09,458.0,9.0,11457.500000,3.0,3.0
...,...,...,...,...,...,...,...
122545,55.0,1.920940e+09,325.0,4.0,8128.333333,3.0,3.0
122546,56.0,1.920940e+09,327.0,22.0,8193.333333,3.0,1.0
122547,139.0,1.920940e+09,329.0,0.0,8225.000000,0.0,0.0
122548,54.0,1.920940e+09,331.0,20.0,8291.666667,3.0,2.0


In [ ]:
#Da Mattia
def event_plot(event):
    
    fig,ax1 = plt.subplots(figsize=(1.5*4.2*17,1.5*1.3*6))
    
    #dimensions of cells
    dx=42
    dy=13
    
    x = []
    y = []
    for i in range(len(event)):
        hit = event.iloc[i]
        y1 = dy * (4 - hit.LAYER + 1/2)
        y.append(y1)
        y.append(y1)
        if hit.LAYER%2 == 0:
            x1 = dx * (hit.COLUMN - 1/2) + hit.X 
            x2 = dx * (hit.COLUMN - 1/2) - hit.X 
            x.append(x1)
            x.append(x2)
        else:
            x1 = dx * hit.COLUMN + hit.X 
            x2 = dx * hit.COLUMN - hit.X 
            x.append(x1)
            x.append(x2)
            

    #draw the detector
    ax1.set_xlim(0,42*16 + 42)
    ax1.set_ylim(-13,13*4 + 13)


    #coordinates of layer 4
    x_1=np.arange(1,17)*42-dx
    y_1=0
    
    #coordinates of layer 3
    x_2=x_1+dx/2
    y_2=dy
    
    #coordinates of layer 2
    x_3=x_1
    y_3=2*dy
    
    #coordinates of layer 1
    x_4=x_2
    y_4=3*dy
    
    for i in range(16):
        rect = patches.Rectangle((x_1[i],y_1),dx,dy,linewidth=10,edgecolor='black',facecolor='none')
        ax1.add_patch(rect)
    
        rect = patches.Rectangle((x_2[i],y_2),dx,dy,linewidth=10,edgecolor='black',facecolor='none')
        ax1.add_patch(rect)
    
        rect = patches.Rectangle((x_3[i],y_3),dx,dy,linewidth=10,edgecolor='black',facecolor='none')
        ax1.add_patch(rect)
        
        rect = patches.Rectangle((x_4[i],y_4),dx,dy,linewidth=10,edgecolor='black',facecolor='none')
        ax1.add_patch(rect)
        
        ax1.axis('off')
        
    ax1.scatter(x,y, marker = 'x', c = 'red', s = 500, linewidth = 7)
    return ax1

In [ ]:
#da Mattia. Nota: dovrebbe essere solido sia all'ordine 0 che 2
for i in np.unique(data_139prova.ORBIT_CNT.values)[:10]:
    try:
        event = data_139prova.groupby(['ORBIT_CNT','EVENT']).get_group((i,1))
        event_plot(event)
    except:
        event = data_139prova.groupby(['ORBIT_CNT']).get_group(i)
        event_plot(event)